In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop2.7.tgz
!tar xf spark-3.2.2-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.2-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://big-data-challenge-bucket.s3.eu-west-2.amazonaws.com/amazon_reviews_us_Digital_Video_Games_v1_00.tsv" 
spark.sparkContext.addFile(url)
video_games_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv"), sep='\t', header=True, inferSchema=True, timestampFormat="yyyy-mm-dd")

# Show DataFrame
video_games_reviews_df.show()

# 1. Count the number of records (rows) in the dataset.

In [ ]:
print(f'There are {video_games_reviews_df.count()} rows in this datatset.')

In [ ]:
print(f'There are {video_games_reviews_df.count()} rows and {len(video_games_reviews_df.columns)} columns in this datatset.')

# Transform the Data

In [ ]:
video_games_reviews_df.printSchema()

In [ ]:
# Be sure that the DataFrames match in data type and in column name.
from pyspark.sql.types import * 
video_games_reviews_df = video_games_reviews_df.withColumn("customer_id", video_games_reviews_df["customer_id"].cast(IntegerType()))\
                          .withColumn("product_parent", video_games_reviews_df["product_parent"].cast(IntegerType()))\
                          .withColumn("star_rating", video_games_reviews_df["star_rating"].cast(IntegerType()))\
                          .withColumn("helpful_votes", video_games_reviews_df["helpful_votes"].cast(IntegerType()))\
                          .withColumn("total_votes", video_games_reviews_df["total_votes"].cast(IntegerType()))\
                          .withColumn("review_date", video_games_reviews_df["review_date"].cast(DateType()))

In [ ]:
video_games_reviews_df.printSchema()

## Create the "review_id_table".

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = video_games_reviews_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

In [ ]:
# Check the scheme of the review_id table
review_id_df.printSchema()
     

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = video_games_reviews_df.select(["product_id","product_title"])

# This table will contain only unique values
products_df = products_df.dropDuplicates()
products_df.show()

In [ ]:
# Check the schema of the products table
products_df.printSchema()

## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = video_games_reviews_df.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

In [ ]:
# Check the customers table schema 
customers_df.printSchema()

In [ ]:
# Change customer_count to integer type
customers_df =customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.printSchema()

## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
from pyspark.sql.functions import col

vine_df = video_games_reviews_df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_df.show()   

In [ ]:
vine_df.printSchema()

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)